# Predicción del precio de Bitcoin

## Bitcoin (apuntes técnicos)
- Definición: criptomoneda descentralizada que opera sobre una red P2P; liquidación y registro en cadena de bloques (blockchain).
- Origen: 2009, seudónimo Satoshi Nakamoto.
- Blockchain: libro mayor distribuido; transacciones agrupadas en bloques enlazados; inmutabilidad práctica mediante encadenamiento criptográfico.
- Propiedades relevantes:
  - Descentralización: ausencia de autoridad central.
  - Pseudonimato: transacciones públicas; identidades no necesariamente reveladas.
  - Escasez programada: tope de emisión ≈ 21 millones de BTC.
  - Seguridad: sustentada en criptografía y consenso distribuido.

Objetivo del cuaderno: construcción de un modelo RNN simple para predicción del precio de cierre de BTC a partir de datos históricos.

Fuente de datos (histórico):
- CoinMarketCap: https://coinmarketcap.com/es/currencies/bitcoin/historical-data/
- Archivo de ejemplo utilizado: https://gist.github.com/mevangelista-alvarado/6f4f28c00f9e683108637cb3c8d5db89

### Lectura del CSV y construcción del DataFrame

In [ ]:
import pandas as pd

_df = pd.read_csv("https://gist.githubusercontent.com/mevangelista-alvarado/6f4f28c00f9e683108637cb3c8d5db89/raw/1ce753283fce23eeab952fa1d660a7069bbe9104/Bitcoin_1_1_2024-6_9_2024_historical_data_coinmarketcap.csv", delimiter=';')
# Ordenamos correctamente
_df = _df.sort_index(ascending=False)
_df.head()

Columnas disponibles en el DataFrame origen:

'timeOpen', 'timeClose', 'timeHigh', 'timeLow', 'name', 'open', 'high',
'low', 'close', 'volume', 'marketCap', 'timestamp'

Selección de columnas de interés

In [ ]:
df = _df[['timeOpen', 'open', 'high', 'low', 'close']]

Conversión a matriz NumPy (variable objetivo: `close`)

In [ ]:
dates = df[['close']].values

### Escalamiento de valores
Se aplica `MinMaxScaler` para normalizar en [0, 1].

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dates)

### Construcción de secuencias (ventana temporal)
Se crean secuencias deslizantes de longitud fija para entrenamiento de la RNN.

Definición del tamaño de ventana temporal (`window_size`)

In [ ]:
window_size = 60

Interpretación: número de pasos de historia usados para predecir el siguiente valor.

Creación de secuencias de datos

In [ ]:
import numpy as np

def create_sequences(data, window_size):
    sequences = []
    labels = []
    for i in range(len(data) - window_size):
        sequences.append(data[i:i+window_size])
        # El precio se ubica en la primera columna
        labels.append(data[i + window_size, 0])
    return np.array(sequences), np.array(labels)

X, y = create_sequences(scaled_data, window_size)

Resultado: `X` contiene ventanas de longitud `window_size`; `y` contiene el valor objetivo siguiente a cada ventana.

### Conjuntos de entrenamiento y prueba
Se realiza partición temporal: 80% entrenamiento, 20% prueba.

In [ ]:
split = int(len(X) * 0.8)
X_train, y_train = X[:split], y[:split]
X_test, y_test = X[split:], y[split:]

### Modelo RNN (arquitectura)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, SimpleRNN

model = Sequential()
model.add(SimpleRNN(units=120, return_sequences=True, input_shape=(window_size, X_train.shape[2])))
model.add(SimpleRNN(units=60, return_sequences=False))
model.add(Dense(units=30))
model.add(Dense(units=1))

Descripción resumida del modelo:
- `SimpleRNN(120, return_sequences=True)` para capturar dependencias temporales.
- `SimpleRNN(60, return_sequences=False)` para resumir la secuencia.
- Capas `Dense` para la salida escalar final (`close`).

### Compilación y entrenamiento

In [ ]:
from tensorflow.keras.optimizers import Adam

learning_rate = 0.001
adam_optimizer = Adam(learning_rate=learning_rate)

# Compilar el modelo
model.compile(optimizer=adam_optimizer, loss='mean_squared_error')

# Entrenar el modelo
model.fit(X_train, y_train, batch_size=1, epochs=10)

### Evaluación en conjunto de prueba

In [ ]:
# Predicciones sobre el conjunto de prueba
predictions = model.predict(X_test)

# Desescalar los datos predichos
predictions = scaler.inverse_transform(np.concatenate((predictions, np.zeros((predictions.shape[0], 3))), axis=1))[:,0]
y_test = scaler.inverse_transform(np.concatenate((y_test.reshape(-1, 1), np.zeros((y_test.shape[0], 3))), axis=1))[:,0]

### Métricas de error

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mean_squared_error(y_test, predictions))

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")

### Visualización de resultados

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

valid = df[split:]
# Reiniciar el índice del DataFrame de validación
valid = valid.reset_index(drop=True)
# Inicializar la columna de predicciones con NaN
valid['Predictions'] = np.nan

# Añadir las predicciones al DataFrame de validación desde el punto donde inician las predicciones
valid.loc[window_size:, 'Predictions'] = predictions
dates_valid = pd.to_datetime(valid['timeOpen']).apply(lambda x: x.strftime('%Y-%m-%d')).tolist()

plt.figure(figsize=(16,8))
plt.title('Modelo RNN para predicción de cierre de BTC')
plt.xlabel('Fecha')
plt.ylabel('Precio de cierre (USD)')
plt.plot(dates_valid, valid[['close', 'Predictions']])
plt.legend(['Valor Real', 'Predicciones'], loc='lower right')
plt.xticks(rotation=90)
plt.show()

### Predicción de valores futuros
Se ilustra el procedimiento para generar predicciones autorregresivas para los próximos `d` días (ejemplo: 10).

In [ ]:
# Generación de secuencias para los siguientes días (predicción autorregresiva)
future_sequences = []
# Última secuencia disponible del conjunto
last_sequence = X[-1]

days = 10
for _ in range(days):
    # Predecir el siguiente valor
    next_value = model.predict(np.array([last_sequence]))[0, 0]

    # Actualizar la secuencia para la siguiente predicción
    last_sequence = np.concatenate((last_sequence[1:], [[next_value]]), axis=0)

    # Añadir la nueva secuencia a las secuencias futuras
    future_sequences.append(last_sequence)

# Conversión de las secuencias futuras a formato compatible con la RNN
future_sequences = np.array(future_sequences)
future_sequences = np.reshape(future_sequences, (future_sequences.shape[0], future_sequences.shape[1], 1))

# Predicciones para los siguientes días
future_predictions = model.predict(future_sequences)

# Desescalado de las predicciones futuras
future_predictions = scaler.inverse_transform(np.concatenate((future_predictions, np.zeros((future_predictions.shape[0], 3))), axis=1))[:,0]

### Observaciones finales
- El experimento ilustra una aplicación de RNN a series temporales financieras (precio de cierre de BTC).
- La utilidad práctica depende del rendimiento fuera de muestra y de la estabilidad temporal de los patrones aprendidos.
- Las predicciones están sujetas a alta incertidumbre; se recomienda complementar con validaciones adicionales, análisis de sensibilidad y comparación con baselines (p. ej., naïve, ARIMA, LSTM).